# Setup

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import datetime
import math
import random
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import torchnlp.nn as nnnlp
from torchvision import datasets, transforms, models
from sklearn.metrics import roc_auc_score
from sklearn.metrics import pairwise
from sklearn import metrics
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from os import listdir
from sys import getsizeof
import os
import nltk, re, pprint
import nltk.tokenize as tk
from collections import Counter
import time
import ast
import sentencepiece as spm
import seaborn as sns
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


import matplotlib.pyplot as plt


E:\Program Files\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
E:\Program Files\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
E:\Program Files\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  stacklevel=1)
E:\Program Files\Anaconda3\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.1.0' currently installed).
  warnings.warn(msg, UserWarning)


In [56]:
help(spm)

Help on module sentencepiece:

NAME
    sentencepiece

DESCRIPTION
    # This file was automatically generated by SWIG (http://www.swig.org).
    # Version 3.0.10
    #
    # Do not make changes to this file unless you know what you are doing--modify
    # the SWIG interface file instead.

CLASSES
    builtins.object
        SentencePieceProcessor
        SentencePieceTrainer
    
    class SentencePieceProcessor(builtins.object)
     |  Methods defined here:
     |  
     |  DecodeIds(self, ids)
     |  
     |  DecodeIdsAsSerializedProto(self, ids)
     |  
     |  DecodePieces(self, pieces)
     |  
     |  DecodePiecesAsSerializedProto(self, pieces)
     |  
     |  EncodeAsIds(self, input)
     |  
     |  EncodeAsPieces(self, input)
     |  
     |  EncodeAsSerializedProto(self, input)
     |  
     |  GetPieceSize(self)
     |  
     |  GetScore(self, id)
     |  
     |  IdToPiece(self, id)
     |  
     |  IsControl(self, id)
     |  
     |  IsUnknown(self, id)
     |  
     

# Miscellaneous Helper Functions

# Reading and loading files

In [77]:
file = []
for i in range(2, 6):
    f = open('00' + str(i) + '.txt', 'r', encoding='UTF-8')
    lines = f.readlines()
    for line in lines:
        file.append(line.strip('\n'))
    print('Appended all lines from 00' + str(i) + ".txt")
    f.close()

Appended all lines from 002.txt
Appended all lines from 003.txt
Appended all lines from 004.txt
Appended all lines from 005.txt


In [83]:
maxlen = 0
def printifmaxlen(s):
    global maxlen
    if(isinstance(s, float)):
        print("float found: " + str(s))
        
    elif len(s)> maxlen:
        
        print(len(s))
        maxlen = len(s)
#wiki_sentences.txt
# lines = pd.Series(file)
# lines = lines[~lines.isna()]
# lines = lines.loc[lines!='+++$+++']
# lines.to_csv(r'wiki_sentences.txt', header=False, index=None, encoding='UTF-8', sep='\n')
lines = pd.read_csv(r'wiki_sentences.txt', header=None, encoding='UTF-8', sep='\n')
lines

,0
0,Propaganda is a concerted set of messages aime...
1,"Instead of impartially providing information, ..."
2,The most effective propaganda is often complet...
3,The desired result is a change of the cognitiv...
4,The word originates from the Latin name Congre...
...,...
424102,Pakistan does generally allow Indian citizens ...
424103,There are concerns that extensive deforestatio...
424104,There are also concerns that the Indus river m...
424105,"On numerous occasions, sediment clogging owing..."


# Dataloader definition and setup

### Data prep

In [ ]:
data_dir = '/data/train'
def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=64)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=64)
    return trainloader, testloader
trainloader, testloader = load_split_train_test(data_dir, .2)
print(trainloader.dataset.classes)

###     Definition

### Assignment and quality assurance

# Model Architecture

# Training loop

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test = 0
def train(model, train_loader, test_loader, 
          learning_rate=0.005, num_epoch=10, print_every=100):
    # Training steps
    start_time = time.time()
    model.train()
    best_auc = [0, 0]
    loss_fn = nn.CrossEntropyLoss()
    loss_fn2 = nn.CrossEntropyLoss()
    loss_fns = [loss_fn, loss_fn2]
    acc = 0
    accs =[[], []]
    aucs = [[], []]
    losses = [[], []]
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print('Beginning training for {} epochs'.format(num_epoch))
    for epoch in range(num_epoch):
        model.train()
        
        for i, (data, labels) in enumerate(train_loader):
            outputs = list(model(data))
            model.zero_grad()
            loss = []
            loss_str = ""
            #print(outputs)


            for j in range(len(outputs)):
                print("calculating loss")
                if len(outputs[j].shape) < 2:
                    outputs[j] = outputs[j].unsqueeze(0)
                #print(loss_fns)
                loss.append(loss_fns[j](outputs[j].float(), labels[j].long().cuda()))
                if(j <len(outputs)-1):
                    loss[j].backward()
                else:
                    loss[j].backward()
                loss_str += "Last Loss {}: {:6.4f} |".format(j, loss[j].item())
            
                
                ##for each output, run the corresponding loss function with the corresponding label
                
            
#             for output in outputs:
#                 if(len(output.shape)<2):
#                     output = output.unsqueeze(0)
#             output0 = outputs[0]
#             output1 = outputs[1]

#             loss = loss_fn(output0.float(), y0.long())#labels.long())
#             loss2 = loss_fn2(output1.float(), y1.long())#labels.long())

#             loss.backward()
#             loss2.backwards()
            optimizer.step()
            

            if (i + 1) % print_every == 0:
                print('Train set | epoch: {:3d}/{} | {:6d}/{:6d} batches |{} Elapsed Time: {:>9}'.format(
                    epoch + 1, num_epoch, i + 1, len(train_loader), loss_str, time.strftime('%H:%M:%S', time.gmtime(int((time.time() - start_time))))))     
        correct = [0, 0]
        total = 0
        model.eval()

        predictions = [ [], []]
        truths = [[], []]

        full_output = []

        with torch.no_grad():
            for i, (data, labels) in enumerate(test_loader):
                outputs = list(model(data))
#                pred = []
                for j in range(len(outputs)):#for each of the outputs
                    while(len(outputs[j].shape)<2):##make sure each output has shape length 2
                        outputs[j] = outputs[j].unsqueeze(0)
                    if(outputs[j].shape[-1] <=2):##if output is binary, take the max along that dimension
                        predictions[j].append(torch.max(outputs[j], 1)[1])
                        predictions[j] += list(pred[j].cpu().numpy())
                        correct[j] += (pred[j].cpu() == labels[j].squeeze().long().cpu()).sum()


                    else:##otherwise, using the maximums as the indices, generate an array of the same shape to the output

                        idx = torch.max(outputs[j], 1)[1]
                        preds = np.zeros_like(outputs.cpu().numpy())
                        for i in range(preds.shape[0]):
                            preds[i, idx[i]] = 1
                        
                        predictions[j].append(preds)
                        correct[j] += (idx.cpu() == labels[j].squeeze().long().cpu()).sum()

                        
                    #print(pred)
                    truths[j] += list(labels[j].cpu().numpy().flatten())
                    total += labels[0].size(0)


#                 pred = []
#                 pred.append(torch.max(output[0], 1)[1])
#                 pred.append(torch.max(output[1], 1)[1])
#                 #print(output, pred)
#                 #print(output.shape)
#                 #output = nn.Softmax(dim=1)(output)
#                 full_output += list(output[0].cpu().numpy())
#                 predictions[0] += list(pred[0].cpu().numpy())
#                 truths[0] += list(labels[0].cpu().numpy().flatten())
#                 correct[0] += (pred[0].cpu() == labels[0].squeeze().long().cpu()).sum()
#                 predictions[1] += list(pred[1].cpu().numpy())
#                 truths[1] += list(labels[1].cpu().numpy().flatten())
#                 correct[1] += (pred[1].cpu() == labels[1].squeeze().long().cpu()).sum()
               
            #print(truths)
            #print(predictions)
#            acc= [[], []]
            for j in range(len(outputs)):
                if(outputs[j].shape[-1] >=2):
                    preds[j] = np.concatenate(preds[j], axis=1)
            acc_str = ""
            auc_str = ""
            for j in range(len(outputs)):
                
                accs[j].append(100.0 * correct[j] / total)
                #print(accs[j][len(accs[j])-1])
                acc_str += "Accuracy {}: {:6.4f} |".format(j, accs[j][len(accs[j])-1].item())
                if(outputs.shape[-1] <= 2):
                    aucs[j].append(metrics.roc_auc_score(truths[j], predictions[j]))
                else:
                    aucs[j].append(metrics.roc_auc_score(truths[j], predictions[j], multi_class = "ovo"))
                auc_str += "AUC {}: {:6.4f} |".format(j, aucs[j][len(aucs[j])-1].item())

                losses[j].append(loss[j])
                #accs[j].append(acc)
            #print(full_output)
            full_output = np.array(full_output)#.tolist()
            #preds = np.zeros_like(full_output)
#             for i in range(len(predictions)):
#                 preds[i, predictions[i]]=1
# #            print(predictions, "\n", preds)

            #print(full_output.shape)
#             auc = []
# #             for i in range(len(labels)):
#                           metrics.roc_auc_score(truths, predictions)#, multi_class = "ovo")
#             aucs.append(auc)
            if(aucs[0][len(aucs[0])-1] > best_auc[0]):
                best_auc[0] = aucs[0][len(aucs[0])-1]
                best_model_wts = model.state_dict()
#            cm = metrics.confusion_matrix(truths[0], predictions)
            elapse = time.strftime('%H:%M:%S', time.gmtime(int((time.time() - start_time))))
            print('Test set Label 0 | Epoch: {} |{} |{} |{} | Time elapsed: {:>9} | Sample output: {}'.format(
               epoch + 1, acc_str, auc_str, loss_str, elapse, outputs[0][0,:]))
 #           print(cm)
            

            #print('Test set | Epoch: {} | Accuracy: {:4.2f} | Time Elapsed: {:>9}'.format(epoch+1, 
             #   acc, elapse))
        
    model.load_state_dict(best_model_wts)
    return model, predictions, truths, accs, losses, aucs ##save this for later

# Run the model

In [16]:
import gc
gc.collect()
#torch.cuda.empty_cache()
CUDA_LAUNCH_BLOCKING=0
BATCH_SIZE = 32
torch.manual_seed(111)
ntokens = 32000
emsize = 24 # embedding dimension
nhid = 50 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 8 # the number of heads in the multiheadattention models
dropout = 0.5 # the dropout value
nclasses = 2

model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, nclasses, dropout, insize=docNum * 100)
print(model.fc2)
model = model.float().cuda()
model, predictions, truths, accs, losses, aucs = train(model, train_loader=train_loader, test_loader=test_loader, learning_rate = 0.1e-2,num_epoch=200)

Linear(in_features=4, out_features=2, bias=True)


RuntimeError: CUDA error: device-side assert triggered

# Evaluation

In [13]:
truths = [0, 1, 2, 3, 4]
preds = np.ones([5, 5])
preds = preds -.81
for truth in truths:
    preds[truth, truth] = .24
    
metrics.roc_auc_score(truths, preds, multi_class = "ovo")

1.0

# Saving

In [64]:
name = "rflip4"
def save():
    with open(name + '-accs-losses-AUCS-6-25-20.csv', 'w') as f:
        f.write("EPOCH, ACCURACY, LOSS, AUC\n")
        for i in range(len(losses)):
            f.write("{}, {}, {}, {}\n".format(i, accs[i], losses[i], aucs[i]))
    f.close()
    with open(name + '-preds-truths-6-25-20.csv', 'w') as f:
        f.write("PREDICTION, TRUTH\n")
        for i in range(len(losses)):
            f.write("{}, {}\n".format(predictions[i], truths[i]))
    f.close()
    torch.save(model.state_dict(), r'E:/Documents/JohnsonLab/' + name + r'-wts-07-03-20.pth')
save()